In [1]:
# Importing Libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Read csv and create DataFrame
data_csv = pd.read_csv("../data/flights.csv")
df = pd.DataFrame(data_csv).set_index("id")

In [3]:
# Create column descriptions
columns_legend = {
    "id": "Unique identifier for each flight record",
    "year": "Year the flight was recorded (Gregorian calendar)",
    "month": "Numerical value of the month the flight occurred (1-12)",
    "day": "Day of the month when the flight occurred (1-31)",
    "dep_time": "Actual departure time (24-hour format, hhmm)",
    "sched_dep_time": "Scheduled departure time (24-hour format, hhmm)",
    "dep_delay": "Minutes difference between scheduled and actual departure (positive for delay, negative for early departure)",
    "arr_time": "Actual arrival time (24-hour format, hhmm)",
    "sched_arr_time": "Scheduled arrival time (24-hour format, hhmm)",
    "arr_delay": "Minutes difference between scheduled and actual arrival (positive for delay, negative for early arrival)",
    "carrier": "Two-letter airline carrier code (e.g., AA for American Airlines)",
    "flight": "Flight number assigned to the flight",
    "tailnum": "Unique aircraft tail number",
    "origin": "Three-letter code of the departure airport",
    "dest": "Three-letter code of the destination airport",
    "air_time": "Total time in the air, in minutes (excludes ground time)",
    "distance": "Distance traveled from origin to destination, in miles",
    "hour": "Hour of scheduled departure time (24-hour format)",
    "minute": "Minute of scheduled departure time",
    "time_hour": "Full timestamp for scheduled departure (yyyy-mm-dd hh:mm:ss, 24-hour format)",
    "name": "Full name of the airline carrier",
}

# Display legend as dataframe
df_cl = pd.DataFrame(
    list(columns_legend.items()), columns=["Column Name", "Description"]
)
df_cl = df_cl.style.set_properties(**{"text-align": "left"})
df_cl

,Column Name,Description
0,id,Unique identifier for each flight record
1,year,Year the flight was recorded (Gregorian calendar)
2,month,Numerical value of the month the flight occurred (1-12)
3,day,Day of the month when the flight occurred (1-31)
4,dep_time,"Actual departure time (24-hour format, hhmm)"
5,sched_dep_time,"Scheduled departure time (24-hour format, hhmm)"
6,dep_delay,"Minutes difference between scheduled and actual departure (positive for delay, negative for early departure)"
7,arr_time,"Actual arrival time (24-hour format, hhmm)"
8,sched_arr_time,"Scheduled arrival time (24-hour format, hhmm)"
9,arr_delay,"Minutes difference between scheduled and actual arrival (positive for delay, negative for early arrival)"


In [4]:
# Give columns_legend to df as attributes
df.attrs = columns_legend

## DataFrame Exploration
- Summarize & Visualize Data
- Identify Patterns
- Find Relationships
- Note Potential Anomalies

## Exploration Scope
We want to know two things:
- What causes flight delays
- How can we better predict delays

In [5]:
# Examine df
df.attrs

{'id': 'Unique identifier for each flight record',
 'year': 'Year the flight was recorded (Gregorian calendar)',
 'month': 'Numerical value of the month the flight occurred (1-12)',
 'day': 'Day of the month when the flight occurred (1-31)',
 'dep_time': 'Actual departure time (24-hour format, hhmm)',
 'sched_dep_time': 'Scheduled departure time (24-hour format, hhmm)',
 'dep_delay': 'Minutes difference between scheduled and actual departure (positive for delay, negative for early departure)',
 'arr_time': 'Actual arrival time (24-hour format, hhmm)',
 'sched_arr_time': 'Scheduled arrival time (24-hour format, hhmm)',
 'arr_delay': 'Minutes difference between scheduled and actual arrival (positive for delay, negative for early arrival)',
 'carrier': 'Two-letter airline carrier code (e.g., AA for American Airlines)',
 'flight': 'Flight number assigned to the flight',
 'tailnum': 'Unique aircraft tail number',
 'origin': 'Three-letter code of the departure airport',
 'dest': 'Three-lette

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336776 entries, 0 to 336775
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            336776 non-null  int64  
 1   month           336776 non-null  int64  
 2   day             336776 non-null  int64  
 3   dep_time        328521 non-null  float64
 4   sched_dep_time  336776 non-null  int64  
 5   dep_delay       328521 non-null  float64
 6   arr_time        328063 non-null  float64
 7   sched_arr_time  336776 non-null  int64  
 8   arr_delay       327346 non-null  float64
 9   carrier         336776 non-null  object 
 10  flight          336776 non-null  int64  
 11  tailnum         334264 non-null  object 
 12  origin          336776 non-null  object 
 13  dest            336776 non-null  object 
 14  air_time        327346 non-null  float64
 15  distance        336776 non-null  int64  
 16  hour            336776 non-null  int64  
 17  minute         

### Insights from df.info()

Right away we can see that these columns have missing data.
This was most likely cause by canceled flights, error in data entry, or a just a simple reporting issue

dep_time        328521 non-null  float64
dep_delay       328521 non-null  float64
arr_time        328063 non-null  float64
arr_delay       327346 non-null  float64
tailnum         334264 non-null  object
air_time        327346 non-null  float64

### Follow up Query

What (if any) patterns are there in the missing data? Such as certain airline, specific times of year, weather patterns, ect.

In [7]:
df.shape

(336776, 20)

In [8]:
df.isnull().sum()

year                 0
month                0
day                  0
dep_time          8255
sched_dep_time       0
dep_delay         8255
arr_time          8713
sched_arr_time       0
arr_delay         9430
carrier              0
flight               0
tailnum           2512
origin               0
dest                 0
air_time          9430
distance             0
hour                 0
minute               0
time_hour            0
name                 0
dtype: int64

In [9]:
df.sample(n=5)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
id,,,,,,,,,,,,,,,,,,,,
94194,2013,12,12,1811.0,1815,-4.0,2100.0,2100,0.0,DL,2065,N340NW,JFK,ATL,112.0,760,18,15,2013-12-12 18:00:00,Delta Air Lines Inc.
307887,2013,8,30,1057.0,1055,2.0,1346.0,1347,-1.0,DL,1275,N3738B,JFK,SLC,262.0,1990,10,55,2013-08-30 10:00:00,Delta Air Lines Inc.
43175,2013,10,18,845.0,850,-5.0,1150.0,1141,9.0,B6,189,N613JB,JFK,SAN,339.0,2446,8,50,2013-10-18 08:00:00,JetBlue Airways
81791,2013,11,29,1001.0,950,11.0,1126.0,1123,3.0,FL,160,N987AT,LGA,CAK,68.0,397,9,50,2013-11-29 09:00:00,AirTran Airways Corporation
105063,2013,12,24,1724.0,1730,-6.0,2040.0,2058,-18.0,DL,2593,N713TW,JFK,SLC,274.0,1990,17,30,2013-12-24 17:00:00,Delta Air Lines Inc.
